<a href="https://colab.research.google.com/github/aquaqu-v246/AI-Products-All-In-One/blob/main/demo/VibeVoice_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VibeVoice Colab — T4 Quickstart (1.5B)

This notebook provides a quickstart guide to run VibeVoice on Colab with T4. The T4 GPU can only support the 1.5B model due to memory limitations. Please note that T4 can only use SDPA instead of flash_attention_2, which may result in unstable and lower audio quality. For the best TTS experience, we recommend trying the 7B model on a more powerful GPU.


## Step 1: Setup Environment

In [1]:
!nvidia-smi --query-gpu=name,memory.total,driver_version --format=csv,noheader

NVIDIA L4, 23034 MiB, 550.54.15


In [2]:
# Check for T4 GPU
import torch


# Clone the VibeVoice repository
![ -d /content/VibeVoice ] || git clone --quiet --branch main --depth 1 https://github.com/aquaqu-v246/VibeVoice.git /content/VibeVoice
print("✅ Cloned VibeVoice repository")

# Install project dependencies
!uv pip --quiet install --system -e /content/VibeVoice
print("✅ Installed dependencies")

# Download model (~3 minutes)
!HF_XET_HIGH_PERFORMANCE=1 hf download vibevoice/VibeVoice-7B --quiet  --local-dir /content/models/VibeVoice-7B > /dev/null
print("✅ Downloaded model: vibevoice/VibeVoice-7B")


✅ Cloned VibeVoice repository
✅ Installed dependencies
✅ Downloaded model: vibevoice/VibeVoice-7B


## Step 2: Create Transcript

In [3]:
%%writefile /content/my_transcript.txt
Speaker 1: 老王. 你过来看一下.

Speaker 2: 怎么了？神神秘秘的. 你这手里...拿着个鸡蛋？几个意思？最新的接头暗号？

Speaker 1: 你觉得. 这个世界上最完美的食物是什么？

Speaker 2: 那还用问？必须是烧烤配啤酒，火锅配可乐，周五晚上的炸鸡...

Speaker 1: 停. 我说的是，一种独立的、单一的食物.

Speaker 2: 嗯...那...是馒头？不对不对，是米饭？哎呀，你别卖关子了，直接说吧.

Speaker 1: 就是它. 鸡蛋.

Speaker 2: 鸡蛋？ ！就这？我说小丽，你也太没想象力了吧. 虽然它能做的菜挺多，西红柿炒蛋、蛋炒饭、茶叶蛋...但你要说它是“最完美”的食物，那猪肉、牛肉、大虾第一个不服.

Speaker 1: 这就是我们今天要去探索的，一个你每天都可能见到，但从未真正了解过的“神秘组织”.

Speaker 2: 什么“神秘组织”？鸡蛋联合会？

Speaker 1: 我问你个实际的. 为了保持你那“健硕”的身材，你每天吃几个鸡蛋？

Speaker 2: 我这么标准的身材需要保持吗？一天一个呗，有时候忘了也就不吃了. 你呢？仙女是不是只喝露水？

Speaker 1: 我每天吃四个.

Speaker 2: 四...四个？ ！你再说一遍？ four？ ！我的天，你疯了吗？你知不知道有个词叫“胆固醇”？你这是在给自己的血管铺“路障”啊！我妈都说了，一天最多吃一个，蛋黄还得扔一半！

Speaker 1: 你看，我就知道你会是这个反应. 网上那些所谓的“养生常识”，已经像钢印一样刻在你的脑子里了. 但今天，我就要带你进入一个“鸡蛋的平行宇宙”. 在那个世界里，你对鸡蛋的所有“常识”，可能...都是错的.

Speaker 2: 嚯，口气不小啊. 行，我今天就当一回你的“捧哏”，我倒要看看，你怎么把这平平无奇的鸡蛋，说出一朵花来.

Speaker 1: 那我们就从最直观的地方开始. 你去菜市场买鸡蛋，尤其是什么“走地鸡”、“山养鸡”的蛋，老板会怎么跟你推销？

Speaker 2: 他会...他会当着我的面敲开一个，然后指着蛋黄说：“老板你看！我这蛋，蛋黄是红的！有营养！”

Speaker 1: 没错！ “蛋黄颜色越深，营养就越高”. 这是不是一条铁律？

Speaker 2: 难道不是吗？这都是老祖宗传下来的智慧.

Speaker 1: 智慧？呵呵. 如果我告诉你，这个蛋黄的颜色，现在就像女生化妆一样，想要什么颜色，就能给你调出什么颜色，你信吗？

Speaker 2: 啊？什么意思？给鸡也用上“美颜滤镜”了？这怎么可能？难道在蛋壳上打个洞，用注射器把颜料打进去？

Speaker 1: 你的想象力只配用在吃的上. 当然不是这么低级的手段. 答案在鸡的饲料里. 现在的一些饲料生产商，已经可以“精准地”通过在鸡饲料里添加一种东西，来控制蛋黄的颜色. 这种东西叫“合成色素”.

Speaker 2: 等等，这个词我熟，不就是那些饮料、零食里加的玩意儿吗？这...这也用到鸡身上了？

Speaker 1: 不仅用，而且用得非常“科学”. 他们甚至有一本专门的“蛋黄颜色比对卡”，就像油漆店里的色卡一样，上面有足足十五个级别，从嫩黄色到橘红色，养鸡场的老板想要他的蛋黄呈现出哪种颜色，就去买对应色号的饲料添加剂.

Speaker 2: 我的天...这简直是“潘通色卡”出鸡蛋限定版了啊. 也就是说，我们看到的那个诱人的、红彤彤的蛋黄，很可能只是“染”出来的？和营养价值半毛钱关系都没有？

Speaker 1: 在很多情况下，是的. 这就是现代食品工业的“障眼法”. 不仅是蛋黄，就连鸡肉的颜色，都可以被“设计”.

Speaker 2: 别...你可别再吓我了. 鸡肉怎么设计？难道还有“彩虹鸡”？

Speaker 1: 那倒不至于. 但你有没有发现，现在超市里的鸡胸肉，吃起来特别“柴”，一点鸡肉的香味都没有？

Speaker 2: 有有有！我一直以为是我的厨艺问题！每次做鸡胸肉，都感觉像在嚼一块湿木头，不放一大堆调味料根本没法吃.

Speaker 1: 这不是你的问题. 这是因为现在的鸡，它们被培育的目标只有一个：长得快，长得大. 因为鸡是按重量卖的. 至于风味？那不重要. 于是，我们就得到了一块块淡然无味的“蛋白质纤维”.

Speaker 2: 太真实了...

Speaker 1: 那怎么解决这个问题呢？很简单. 你下次去超市，拿起一块包装好的鸡胸肉，仔细看背后的配料表，你很可能会在上面看到两个词：“天然风味”，或者更直接的，“人工风味”.

Speaker 2: 我去！真的假的？鸡肉里加“鸡肉味”？这不跟老婆饼里没老婆，鱼香肉丝里没有鱼一个道理吗？

Speaker 1: 恭喜你，答对了. 他们必须往这块“白板”一样的肉里加东西，才能让它吃起来...更像一块鸡肉. 这就是现实.

Speaker 2: 唉...感觉不会再爱了. 那我们岂不是完全没法判断一个蛋的好坏了？

Speaker 1: 也别这么悲观. 如果抛开这些“科技与狠活”，一个天然的、没有被人工干预的蛋黄，它的颜色的确可以反映出一些营养信息，比如类胡萝卜素的含量. 但重点是，一个鸡蛋的灵魂，完全取决于它的母亲——那只鸡，过的是什么样的生活，吃的是什么样的食物.

Speaker 2: 这话说得，怎么有点文艺...

Speaker 1: 别打岔. 这就引出了下一个大“坑”：鸡蛋包装上的那些“花言巧语”.

Speaker 2: “花言巧语”？你是说那些“散养鸡蛋”、“谷饲鸡蛋”？

Speaker 1: 没错. 就说这个“散养”，英文叫“Free-range”. 你听到这个词，脑子里浮现的是什么画面？

Speaker 2: 一大片绿草地，一群鸡在阳光下快乐地奔跑、啄虫子，呼吸着自由的空气...

Speaker 1: 醒醒. 现实可能是，一大群鸡挤在一个巨大的鸡舍里，只不过它们脚下没有笼子. 而所谓的“通往户外”，可能只是墙角开了一个狗洞大小的口子，大多数鸡一辈子都找不到那个洞口在哪. 更关键的是，这个标准，很多时候根本没有人去现场检查.

Speaker 2: 啊？ ！这不就是“字面意义”上的“散养”吗？只要没关在笼子里就行？这...这是诈骗啊！

Speaker 1: 法律的边缘，不能叫诈骗. 还有那个听起来特别高大上的“A级”鸡蛋. 你猜猜是啥意思？

Speaker 2: 营养是A级？鸡的品种是A级？还是...下蛋的母鸡考试成绩是A？

Speaker 1: “A级”指的是，这个蛋壳，它长得特别规整，表面光滑，没有裂缝. 没了.

Speaker 2: ...就这？就因为长得好看？外貌协会已经渗透到鸡蛋界了吗？我算是明白了，这些词，都是市场部的“杰作”.

Speaker 1: 完全正确. 它们存在的唯一目的，就是让你心甘情愿地多掏钱. 所以，想要找到真正的“好蛋”，你必须学会忽略这些标签，去关注最本质的东西.

Speaker 2: 本质的东西...是什么？

Speaker 1: 是历史. 我给你看个数字，你可能会吓一跳. 2015年的时候，买一只鸡大概要花7美金.

Speaker 2: 7美金...五十块钱人民币不到？挺便宜的啊.

Speaker 1: 是啊，便宜得离谱. 你想，养一只鸡，要给它吃的，要照顾它，最后卖7块钱，怎么赚钱？答案就是用最便宜的谷物饲料，让它在最短的时间里催肥长大. 但你知道吗，在1948年，买一只鸡要多少钱？

Speaker 2: 1948年...那时候物价低，估计...1美金？

Speaker 1: 3美金.

Speaker 2: 3美金？比2015年还贵？

Speaker 1: 如果你考虑到通货膨胀，1948年的3美金，大概相当于今天的30美金！

Speaker 2: 我的妈呀！三百多块钱买一只鸡？ ！

Speaker 1: 为什么那么贵？因为那个时候的鸡，品质和现在是天壤之别. 它是在一个更自然的环境里，花了更长的时间慢慢长大的. 你想想，我们用七十年的时间，把鸡的价格打下来了，但我们失去的是什么？是风味，是营养，是食物本该有的样子.

Speaker 2: 听你这么一说...感觉我们现在吃的，只是一个长得像“鸡”的“蛋白质合成物”.

Speaker 1: 好了，说了这么多“黑暗面”，我们该聊点开心的了. 为什么我冒着“胆固醇超标”的“风险”，每天还要吃四个鸡蛋. 因为它的好处，实在太强大了.

Speaker 2: 好，请开始你的表演. 我现在已经有点信了.

Speaker 1: 首先，也是最重要的一点：蛋白质. 你健身，肯定知道蛋白质很重要. 但蛋白质和蛋白质，是不一样的.

Speaker 2: 哦？还有“三六九等”？

Speaker 1: 当然. 我们衡量一种蛋白质好不好，有一个黄金标准，就是它的“生物转化率”. 说白了，就是你吃下去100克的蛋白质，有多少能真正变成你身体的一部分，比如你的肌肉. 如果把最牛的母乳排除在外，你猜鸡蛋的转化率是多少？

Speaker 2: 我猜...有个百分之二十？

Speaker 1: 百分之四十八！

Speaker 2: 将近一半？ ！这么高？ ！

Speaker 1: 非常高. 我给你个对比你就知道了. 我们平时吃的肉、鱼，转化率大概是百分之三十二. 而现在非常流行的，听起来很健康的“大豆蛋白”，你猜是多少？

Speaker 2: 嗯...比肉低点，百分之二十五？

Speaker 1: 百分之十七.

Speaker 2: 这么低？ ！那健身房里大家疯狂喝的那个“乳清蛋白粉”呢？那个应该很高吧，都是科技的结晶.

Speaker 1: 乳清蛋白，百分之十六.

Speaker 2: 噗...还没大豆蛋白高？ ！那我喝那玩意儿不是交了智商税？ ！

Speaker 1: 所以你看，鸡蛋，这个最古老、最朴素的食物，它的蛋白质效率，秒杀了市面上几乎所有的“高科技”产品. 更有趣的是，如果你只吃蛋清，不吃蛋黄，它的转化率也会掉到百分之十七.

Speaker 2: 又是蛋黄！看来蛋黄才是真正的“MVP”啊！

Speaker 1: 完全正确. 蛋黄和蛋清在一起，才是完整的“梦之队”.

Speaker 2: 好吧，蛋白质这一点，你彻底说服我了. 还有什么“大招”？

Speaker 1: 当然. 我们来聊聊你的脑子.

Speaker 2: 我的脑子怎么了？挺好使的啊，你看我吐槽得多快.

Speaker 1: 想要它一直好使，你就需要一种东西，叫“胆碱”. 它是构成我们大脑神经递质的重要物质，能让你的思维更敏捷，记忆力更好. 而鸡蛋，是胆碱最丰富的来源之一. 同时，胆碱还能帮助你的肝脏代谢脂肪，对预防脂肪肝有奇效.

Speaker 2: 补脑还护肝？这个我需要.

Speaker 1: 然后，我们再来聊聊你的眼睛.

Speaker 2: 眼睛？我眼睛也挺好的啊，5.0的视力.

Speaker 1: 还能好多久？你每天看手机、看电脑多长时间？屏幕发出的蓝光，正在悄悄地伤害你的视网膜. 而蛋黄里有两种神奇的“天然太阳镜”，叫“叶黄素”和“玉米黄质”.

Speaker 2: 这名字我好像在一些保健品的广告里听过.

Speaker 1: 对！但你根本不需要去买那些昂贵的保健品. 这两种物质，是构成我们视网膜黄斑区域的核心色素，它们能像滤镜一样，过滤掉有害的蓝光，保护你的眼睛，预防老年性黄斑变性. 我们全身，眼睛里这两种营养素的浓度是最高的，而鸡蛋，就是它们最好的膳食来源.

Speaker 2: 哇...吃个鸡蛋，等于给眼睛请了个“保镖”？

Speaker 1: 你可以这么理解. 还没完. 鸡蛋里有一种非常重要的脂溶性维生素，维生素K2. 你知道补钙吧？

Speaker 2: 知道啊，喝牛奶，吃钙片.

Speaker 1: 但如果只有钙，没有维生素K2，你补进去的钙，很可能不会跑到你的骨头和牙齿里，而是沉积在你的血管壁上，造成动脉硬化. 维生素K2就像一个“交通警察”，它指挥着钙，让它去该去的地方. 而鸡蛋，同时富含维生素K2和可以促进钙吸收的维生素D，它们是“黄金搭档”.

Speaker 2: 我的天...一个小小的鸡蛋里，怎么藏了这么多“神仙”？

Speaker 1: 这就是我为什么说它很完美. 它就像一个“全能营养工具箱”. 除了上面说的，它还有保护心血管的维生素E，促进生长的维生素A，一大堆B族维生素，Omega-3脂肪酸，各种微量元素...几乎所有人体需要的基础营养素，它都给你配齐了.

Speaker 2: 听完了...我感觉我过去吃的都不是鸡蛋，只是一个“长得像蛋的球”. 我现在就想去吃它四个！不过...怎么吃最好？煎着吃？煮着吃？

Speaker 1: 问到点子上了. 最好的吃法，是那种能让蛋黄保持“溏心”状态的. 比如水波蛋，或者单面煎的太阳蛋. 因为那些宝贵的类胡萝卜素和一些维生素，比较怕高温.

Speaker 2: 那我最爱吃的炒鸡蛋和水煮蛋呢？

Speaker 1: 如果是全熟的水煮蛋，也没问题，大部分营养还在. 如果是炒鸡蛋，我教你一个“减损”小技巧：不要在碗里把鸡蛋完全打散了再下锅. 你应该把鸡蛋直接打进锅里，等蛋清稍微凝固了，再用铲子轻轻搅散，这样能最大程度地保留它的营养.

Speaker 2: 哇哦...连吃法都有这么多讲究. 小丽，我今天真是对你刮目相看，你才是我心中的“蛋神”！

Speaker 1: 哈哈，少拍马屁. 所以，结论就是，别再妖魔化鸡蛋和它的胆固醇了. 对于我们大多数人来说，每天吃上两三个、甚至四个高品质的鸡蛋，得到的好处，远远大于那一点点所谓的“风险”.

Speaker 2: 明白了！彻底明白了！我决定了，以后每天早上都监督你吃四个，我吃三个！周末我们一起去你说的那个“农夫市场”逛逛，看看能不能找到传说中的“神仙鸡蛋”！

Speaker 1: 这还差不多. 好了，听了这么多关于鸡蛋的“秘密”，不知道屏幕前的你有什么想法呢？你平时吃几个鸡蛋？或者你还知道什么关于鸡蛋的冷知识？欢迎在评论区给我们留言互动哦. 如果你觉得今天的内容对你有帮助，千万别忘了点赞、收藏，再点个关注，这样就不会错过我们后面的精彩内容啦！

Speaker 2: 对对对，关注我们，下次小丽可能要讲“一头牛的宇宙”了！

Speaker 1: 你给我闭嘴！


Writing /content/my_transcript.txt


## Step 3: Generate Audio

In [6]:
# Run Python script to generate audio from transcript
!python /content/VibeVoice/demo/inference_from_file.py \
    --model_path /content/models/VibeVoice-7B \
    --txt_path /content/my_transcript.txt \
    --speaker_names Xinran Baowen

# Display audio controls
from IPython.display import Audio
Audio("/content/outputs/my_transcript_generated.wav")


2025-09-13 14:11:51.383116: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-13 14:11:51.400681: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757772711.422745   11258 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757772711.429200   11258 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757772711.445560   11258 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

ValueError: rate must be specified when data is a numpy array or list of audio samples.

# Step 4: Download Audio

In [5]:
from google.colab import files
files.download("/content/outputs/my_transcript_generated.wav")


FileNotFoundError: Cannot find file: /content/outputs/story_generated.wav

In [7]:
from google.colab import runtime
runtime.unassign()   # 相当于在菜单点 “断开并删除运行时”